In [20]:
import random
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
import zipfile
import os

current_directory = os.getcwd()

# Function to generate random problem data
def generate_problem_data():
    problems = []

    # Problem Type 1: Graph-based problems
    for _ in range(4):  # 4 problems of this type
        x2 = random.randint(2, 5)  # Ensure x2 > 0
        y1 = random.randint(-3, -1)  # Negative value
        y2 = random.randint(1, 3)    # Positive value
        if random.choice([True, False]):  # Randomize the order of y1 and y2
            y1, y2 = y2, y1
        problems.append({"type": "graph", "x2": x2, "y1": y1, "y2": y2})


    # Problem Type 2: Table-based problems
    for _ in range(4):  # 4 problems of this type
        times = [0] + sorted(random.sample(range(1, 6), 3))  # Include 0 and 3 more times
        distances = [round(random.uniform(-2, 2), 1) for _ in range(4)]  # Random distances
        while len(set(distances)) < 2:  # Ensure valid rate of change
            distances = [round(random.uniform(-2, 2), 1) for _ in range(4)]
        problems.append({"type": "table", "times": times, "distances": distances})

    # Problem Type 3: Word-based problems
    for _ in range(2):  # 2 problems of this type
        start_floor, end_floor = random.randint(-2, 2), random.randint(-2, 2)
        while start_floor == end_floor:  # Ensure valid rate of change
            start_floor, end_floor = random.randint(-2, 2), random.randint(-2, 2)
        time = random.randint(1, 5)
        problems.append({"type": "word", "start_floor": start_floor, "end_floor": end_floor, "time": time})

    return problems

# Function to generate graphs and LaTeX content
def generate_light_grid_graphs_and_latex(problems, set_name):
    """
    Generate graphs with light grid lines, bolded zero XY axes, and no perimeter bold borders.
    """
    problems_text = ""
    answers_text = ""

    for i, problem in enumerate(problems, 1):
        if problem["type"] == "graph":
            x1, x2 = 0, problem["x2"]
            y1, y2 = problem["y1"], problem["y2"]
            rate_of_change = round((y2 - y1) / (x2 - x1), 2)
            
            # Generate and save the graph
            graph_path = f"{set_name}_problem_{i}.png"
            fig, ax = plt.subplots(figsize=(5, 4))  # Slightly larger figure size
            ax.plot([x1, x2], [y1, y2], marker='o', linestyle='-', color='blue', linewidth=2, markersize=8)
            
            # Titles and labels
            ax.set_title("Distance vs. Time", fontsize=12, weight="bold")
            ax.set_xlabel("Time (s)", fontsize=10, weight="bold")
            ax.set_ylabel("Distance (m)", fontsize=10, weight="bold")
            
            # Hide unnecessary spines
            ax.spines['top'].set_visible(False)
            ax.spines['right'].set_visible(False)
            plt.box(False)
            
            # Bold zero axes
            ax.axhline(0, color="black", linewidth=1.5)
            ax.axvline(0, color="black", linewidth=1.5)

            ax.spines['bottom'].set_position(('data', 0))
            ax.xaxis.set_label_coords(0.5, -0.1)

            # Format ticks and limits
            ax.set_xticks(range(0, x2 + 2))
            ax.set_xticklabels(range(0, x2 + 2), fontsize=9)
            ax.set_yticks(range(min(y1, y2) - 1, max(y1, y2) + 2))
            ax.set_yticklabels(range(min(y1, y2) - 1, max(y1, y2) + 2), fontsize=9)
            
            # Add a light grid for better visibility
            ax.grid(color="gray", linestyle="--", linewidth=0.5, alpha=0.7)
            
            # Add padding and make layout tight
            fig.tight_layout()
            
            # Save and close
            plt.savefig(graph_path, dpi=300)  # Save with high resolution
            plt.close()

            problems_text += (
                f"\\item Calculate the rate of change from the graph below.\n\n"
                f"\\includegraphics[width=0.6\\textwidth]{{{set_name}_problem_{i}.png}}\n\n\n"
                f"\\vspace{{2cm}}  % Add space for students to work\n"
            )
            answers_text += (
                f"\\item The rate of change is $\\frac{{\\Delta y}}{{\\Delta x}} = \\frac{{{y2}-{y1}}}{{{x2}-{x1}}} = {rate_of_change}~\\text{{units/s}}$.\n\n"
            )

        elif problem["type"] == "table":
            times, distances = problem["times"], problem["distances"]
            rate_of_change = round((distances[-1] - distances[0]) / (times[-1] - times[0]), 2)
            problems_text += (
                "\\item Calculate the rate of change from the table below:\n\n"
                "\\begin{tabular}{|c|c|}\n"
                "\\hline\n"
                "Time (s) & Distance (m) \\\\\n"
                "\\hline\n"
            )
            problems_text += "\n".join([f"{t} & {d} \\\\" for t, d in zip(times, distances)]) + "\n"
            problems_text += "\\hline\n\\end{tabular}\n\n\\vspace{2cm}\n"  # Add space
            answers_text += (
                f"\\item The rate of change is $\\frac{{\\Delta y}}{{\\Delta x}} = \\frac{{{distances[-1]}-{distances[0]}}}{{{times[-1]}-{times[0]}}} = {rate_of_change}~\\text{{units/s}}$.\n\n"
            )

        elif problem["type"] == "word":
            start_floor, end_floor, time = problem["start_floor"], problem["end_floor"], problem["time"]
            rate_of_change = round((end_floor - start_floor) / time, 2)
            problems_text += (
                f"\\newpage\\item An elevator starts at floor {start_floor}. After {time} seconds, it is at floor {end_floor}. "
                f"Calculate the rate of change.\n\n\\vspace{{7cm}}\n"
            )
            answers_text += (
                f"\\item The rate of change is $\\frac{{\\Delta y}}{{\\Delta x}} = \\frac{{{end_floor}-{start_floor}}}{{{time}}} = {rate_of_change}~\\text{{floors/s}}$.\n\n"
            )

    latex_content = (
        "\\documentclass[12pt]{article}\n"
        "\\usepackage{graphicx}\n"
        "\\usepackage{amsmath}\n"
        "\\usepackage[margin=1in]{geometry}\n"
        "\\begin{document}\n"
        f"\\section*{{Worksheet {set_name}}}\n"
        "\\begin{enumerate}\n"
        f"{problems_text}"
        "\\end{enumerate}\n"
        "\\newpage\n"
        "\\section*{Answer Key}\n"
        "\\begin{enumerate}\n"
        f"{answers_text}"
        "\\end{enumerate}\n"
        "\\end{document}"
    )

    return latex_content

# Generate problems and updated LaTeX files with proper light grid and bolded axes
problem_set_A_light = generate_problem_data()
problem_set_B_light = generate_problem_data()

latex_A_light = generate_light_grid_graphs_and_latex(problem_set_A_light, "A")
latex_B_light = generate_light_grid_graphs_and_latex(problem_set_B_light, "B")

# Save the updated LaTeX files
latex_A_light_path = "worksheet_A_light.tex"
latex_B_light_path = "worksheet_B_light.tex"

with open(latex_A_light_path, "w") as f:
    f.write(latex_A_light)

with open(latex_B_light_path, "w") as f:
    f.write(latex_B_light)

# Save all associated graph files for A and B with light grid and bold zero axes
graph_files = []
for file in os.listdir(current_directory):
    if file.endswith(".png") and ("A_light_" in file or "B_light_" in file):
        graph_files.append(f"{file}")

# Output paths to all generated files
latex_A_light_path, latex_B_light_path, graph_files

('worksheet_A_light.tex',
 'worksheet_B_light.tex',
 ['B_light_problem_4.png',
  'A_light_problem_4.png',
  'B_light_problem_2.png',
  'B_light_problem_3.png',
  'A_light_problem_1.png',
  'A_light_problem_3.png',
  'B_light_problem_1.png',
  'A_light_problem_2.png'])